# EDA and Linear Regression with Evaluation Metrics on Diamonds Dataset

<a id = "1"></a>
# Importing the Necessary Libraries

In [1]:
import numpy as np 
import pandas as pd
import pandas
#import seaborn as sns
#import matplotlib.pyplot as plt
from collections import Counter
#%matplotlib inline
#import seaborn as sns; sns.set()

#import graphviz 

#from pandas_profiling import ProfileReport

#from scipy.stats import norm, skew, boxcox
from sklearn.model_selection import KFold


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.decomposition import PCA

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV, cross_val_score


from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE

from sklearn import metrics


import warnings
warnings.filterwarnings("ignore")

In [2]:
dataset = pandas.read_csv('/notebooks/notebooks/diamonds.csv', usecols=['carat','cut','color','clarity','depth','table','price','x','y','z'])
dataset.sample(10)

,carat,cut,color,clarity,depth,table,price,x,y,z
1097,1.00,Fair,I,SI1,66.0,56.0,2912,6.31,6.24,4.13
23253,1.78,Premium,H,SI2,59.1,60.0,11262,7.93,7.87,4.67
35981,0.42,Premium,G,VS1,60.9,58.0,921,4.81,4.84,2.94
13849,1.20,Very Good,E,SI2,63.3,55.0,5645,6.77,6.73,4.27
40400,0.34,Ideal,D,VVS1,61.5,56.0,1133,4.47,4.50,2.76
8840,1.01,Ideal,E,SI2,60.9,57.0,4479,6.47,6.51,3.95
41763,0.38,Ideal,E,VVS1,62.4,53.2,1245,4.62,4.66,2.90
32400,0.37,Premium,F,SI1,62.4,59.0,791,4.60,4.56,2.86
19642,1.24,Ideal,H,VS2,60.8,56.0,8257,6.92,6.96,4.22
24338,1.50,Fair,D,VS2,65.4,53.0,12606,7.12,7.10,4.65


In [3]:
dataset.isnull().sum()

carat      0
cut        0
color      0
clarity    0
depth      0
table      0
price      0
x          0
y          0
z          0
dtype: int64

In [4]:
dataset.describe()

,carat,depth,table,price,x,y,z
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,61.749405,57.457184,3932.799722,5.731157,5.734526,3.538734
std,0.474011,1.432621,2.234491,3989.439738,1.121761,1.142135,0.705699
min,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000
max,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


<a id = "7"></a>
# Basic Data Analysis

In [5]:
dataset[["cut","price"]].groupby(["cut"], as_index = False).mean().sort_values(by="price",ascending = False)

,cut,price
3,Premium,4584.257704
0,Fair,4358.757764
4,Very Good,3981.759891
1,Good,3928.864452
2,Ideal,3457.541970


In [6]:
mean_price_by_cut = dataset[["cut","price"]].groupby(["cut"], as_index = False).mean().sort_values(by="price",ascending = False)

<a id = "9"></a>
## Color - Price

In [7]:
dataset[["color","price"]].groupby(["color"], as_index = False).mean().sort_values(by="price",ascending = False)

,color,price
6,J,5323.818020
5,I,5091.874954
4,H,4486.669196
3,G,3999.135671
2,F,3724.886397
0,D,3169.954096
1,E,3076.752475


In [8]:
mean_price_by_color = dataset[["color","price"]].groupby(["color"], as_index = False).mean().sort_values(by="price",ascending = False)

<a id = "10"></a>
## Clarity - Price

In [9]:
mean_price_by_clarity = dataset[["clarity","price"]].groupby(["clarity"], as_index = False).mean().sort_values(by="price",ascending = False)

In [10]:
dataset[["clarity","price"]].groupby(["clarity"], as_index = False).mean().sort_values(by="price",ascending = False)

,clarity,price
3,SI2,5063.028606
2,SI1,3996.001148
5,VS2,3924.989395
0,I1,3924.168691
4,VS1,3839.455391
7,VVS2,3283.737071
1,IF,2864.839106
6,VVS1,2523.114637


**There are 8 different 'Clarity' values. Uneven distribution is not observed.**

<a id = "11"></a>
## Carat - Price

In [11]:
dataset[["carat","price"]].groupby(["carat"], as_index = False).mean().sort_values(by="price",ascending = False)

,carat,price
265,3.51,18701.000000
246,2.67,18686.000000
271,4.50,18531.000000
272,5.01,18018.000000
237,2.57,17841.666667
...,...,...
4,0.24,505.185039
3,0.23,486.143345
2,0.22,391.400000
1,0.21,380.222222


<a id = "12"></a>
## Depth - Price

In [12]:
dataset[["depth","price"]].groupby(["depth"], as_index = False).mean().sort_values(by="price",ascending = False)

,depth,price
159,69.1,8736.000000
172,70.6,8266.666667
170,70.2,7997.333333
143,67.5,7276.200000
171,70.5,6860.000000
...,...,...
13,54.0,1012.000000
17,54.6,1011.000000
3,51.0,945.000000
14,54.2,905.500000


<a id = "13"></a>
## Table - Price

In [13]:
dataset[["table","price"]].groupby(["table"], as_index = False).mean().sort_values(by="price",ascending = False)

,table,price
126,95.0,13387.000000
6,51.6,13203.000000
3,50.0,8995.500000
65,58.5,6499.700000
120,69.0,6272.222222
...,...,...
98,61.8,1262.000000
11,53.1,1146.666667
93,61.3,929.000000
9,52.8,652.500000


<a id = "19"></a>
# Anomaly Detection

In [14]:
#This code is retrieved from here: https://www.kaggle.com/kanncaa1/dataiteam-titanic-eda#Introduction

def detect_outliers(df,features):
    outlier_indices = []
    
    for c in features:
        # 1st quartile
        Q1 = np.percentile(df[c],25)
        # 3rd quartile
        Q3 = np.percentile(df[c],75)
        # IQR
        IQR = Q3 - Q1
        # Outlier step
        outlier_step = IQR * 1.5
        # detect outlier and their indeces
        outlier_list_col = df[(df[c] < Q1 - outlier_step) | (df[c] > Q3 + outlier_step)].index
        # store indeces
        outlier_indices.extend(outlier_list_col)
    
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(i for i, v in outlier_indices.items() if v > 2)
    
    return multiple_outliers

In [15]:
dataset.loc[detect_outliers(dataset,['carat', 'depth', 'table', 'x', 'y', 'z', 'price'])]

,carat,cut,color,clarity,depth,table,price,x,y,z
13992,2.01,Fair,I,I1,55.9,64.0,5696,8.48,8.39,4.71
17196,2.72,Fair,J,I1,68.2,56.0,6870,8.46,8.43,5.76
18067,2.01,Fair,F,I1,58.7,66.0,7294,8.30,8.19,4.84
19346,3.00,Fair,H,I1,67.1,57.0,8044,8.93,8.84,5.97
21758,3.11,Fair,J,I1,65.9,57.0,9823,9.15,9.02,5.98
...,...,...,...,...,...,...,...,...,...,...
24520,1.56,Ideal,G,VS2,62.2,54.0,12800,0.00,0.00,0.00
26243,1.20,Premium,D,VVS1,62.1,59.0,15686,0.00,0.00,0.00
49556,0.71,Good,F,SI2,64.1,60.0,2130,0.00,0.00,0.00
49557,0.71,Good,F,SI2,64.1,60.0,2130,0.00,0.00,0.00


In [16]:
# drop outliers
dataset = dataset.drop(detect_outliers(dataset,['carat', 'depth', 'table', 'x', 'y', 'z', 'price']),axis = 0).reset_index(drop = True)

In [17]:
dataset = dataset.dropna()

In [18]:
dataset = dataset[dataset.x != 0]
dataset = dataset[dataset.y != 0]
dataset = dataset[dataset.z != 0]

In [19]:
(dataset['x'] == 0).sum().sum()

0

In [20]:
(dataset['y'] == 0).sum().sum()

0

In [21]:
(dataset['z'] == 0).sum().sum()

0

<a id = "20"></a>
# Skewness

In [22]:
dataset.agg(['skew', 'kurtosis']).transpose()

,skew,kurtosis
carat,1.036670,0.723671
depth,-0.086166,5.877811
table,0.704607,1.346901
price,1.625238,2.237933
x,0.376546,-0.768652
y,0.608223,4.895099
z,1.595547,49.509723


<a id = "26"></a>
# One-Hot Encoding

In [23]:
cat_var =  ['cut', 'color', 'clarity']

for i in range (0, len(cat_var)):
    print(f'Unique Values for {cat_var[i]}', dataset[f'{cat_var[i]}'].unique())

Unique Values for cut ['Ideal' 'Premium' 'Good' 'Very Good' 'Fair']
Unique Values for color ['E' 'I' 'J' 'H' 'F' 'G' 'D']
Unique Values for clarity ['SI2' 'SI1' 'VS1' 'VS2' 'VVS2' 'VVS1' 'I1' 'IF']


In [24]:
onehotencoder = OneHotEncoder()

In [25]:
one_hot = ['cut', 'color', 'clarity']

for i in range(0, len(one_hot)):
    dataset[f'{one_hot[i]}'] = pd.Categorical(dataset[f'{one_hot[i]}'])
    dummies = pd.get_dummies(dataset[f'{one_hot[i]}'], prefix = f'{one_hot[i]}_encoded')
    dataset.drop([f'{one_hot[i]}'], axis=1, inplace=True)
    dataset = pd.concat([dataset, dummies], axis=1)

In [26]:
dataset

,carat,depth,table,price,x,y,z,cut_encoded_Fair,cut_encoded_Good,cut_encoded_Ideal,...,color_encoded_I,color_encoded_J,clarity_encoded_I1,clarity_encoded_IF,clarity_encoded_SI1,clarity_encoded_SI2,clarity_encoded_VS1,clarity_encoded_VS2,clarity_encoded_VVS1,clarity_encoded_VVS2
0,0.23,61.5,55.0,326,3.95,3.98,2.43,0,0,1,...,0,0,0,0,0,1,0,0,0,0
1,0.21,59.8,61.0,326,3.89,3.84,2.31,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0.23,56.9,65.0,327,4.05,4.07,2.31,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,0.29,62.4,58.0,334,4.20,4.23,2.63,0,0,0,...,1,0,0,0,0,0,0,1,0,0
4,0.31,63.3,58.0,335,4.34,4.35,2.75,0,1,0,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53751,0.72,60.8,57.0,2757,5.75,5.76,3.50,0,0,1,...,0,0,0,0,1,0,0,0,0,0
53752,0.72,63.1,55.0,2757,5.69,5.75,3.61,0,1,0,...,0,0,0,0,1,0,0,0,0,0
53753,0.70,62.8,60.0,2757,5.66,5.68,3.56,0,0,0,...,0,0,0,0,1,0,0,0,0,0
53754,0.86,61.0,58.0,2757,6.15,6.12,3.74,0,0,0,...,0,0,0,0,0,1,0,0,0,0


<a id = "27"></a>
# Train-Test Split

In [27]:
X = dataset.drop(["price"],axis =1)
y = dataset["price"]

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 101)

print(f'Total # of sample in whole dataset: {len(X)}')
print(f'Total # of sample in train dataset: {len(X_train)}')
print(f'Total # of sample in test dataset: {len(X_test)}')

Total # of sample in whole dataset: 53748
Total # of sample in train dataset: 36011
Total # of sample in test dataset: 17737


In [29]:
sc=StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

<a id = "28"></a>
# Linear Regression

<a id = "29"></a>
## Hyper-Parameter Optimization

In [30]:
LinearRegression().get_params().keys()

dict_keys(['copy_X', 'fit_intercept', 'n_jobs', 'normalize', 'positive'])

In [31]:
parameters = {'fit_intercept': ['True', 'False'],
             'copy_X': ['True', 'False']
             }

gcv = GridSearchCV(LinearRegression(), parameters, cv=15, verbose = 1, n_jobs = -1).fit(X_train, y_train)
print(f'GridSearchView Best Score: {gcv.best_score_*100}')
print(f'GridSearchView Best Estimator: {gcv.best_estimator_}')
print(f'GridSearchView Best Params: {gcv.best_params_}')

Fitting 15 folds for each of 4 candidates, totalling 60 fits
GridSearchView Best Score: 92.26738240745381
GridSearchView Best Estimator: LinearRegression(copy_X='True', fit_intercept='True')
GridSearchView Best Params: {'copy_X': 'True', 'fit_intercept': 'True'}


<a id = "30"></a>
## Model

In [32]:
model_lr = LinearRegression(copy_X='True', fit_intercept='True', n_jobs = -1)
model_lr.fit(X_train, y_train)
pred = model_lr.predict(X_test)
 
train_score = model_lr.score(X_train, y_train)
print(f'Train score of trained model: {train_score*100}')

test_score = model_lr.score(X_test, y_test)
print(f'Test score of trained model: {test_score*100}')

Train score of trained model: 92.4792526824141
Test score of trained model: 91.86644661952487


<a id = "31"></a>
## Evaluation Metrics

In [33]:
print("Explained Variance:",metrics.explained_variance_score(y_test, pred))

Explained Variance: 0.9186652872943553


In [34]:
print("Max Error:",metrics.max_error(y_test, pred))

Max Error: 31276.00080626523


In [35]:
print("Mean Squared Error:",metrics.mean_squared_error(y_test, pred))

Mean Squared Error: 1265301.981993359


In [36]:
print("Mean Absolute Error:",metrics.mean_absolute_error(y_test, pred))

Mean Absolute Error: 720.4312196999747


In [37]:
print("Median Absolute Error:",metrics.median_absolute_error(y_test, pred))

Median Absolute Error: 498.6575598041354


In [38]:
print("R^2:",metrics.r2_score(y_test, pred))

R^2: 0.9186644661952487


In [39]:
n=X_test.shape[0]
p=X_test.shape[1] - 1
R2 = metrics.r2_score(y_test, pred)

adj_rsquared = 1 - (1 - R2) * ((n - 1)/(n-p-1))
print('Adjusted R Squared: {}'.format(adj_rsquared))

Adjusted R Squared: 0.918549656848226


In [40]:
print("Root Mean Squared Error:",np.sqrt(metrics.mean_squared_error(y_test, pred)))

Root Mean Squared Error: 1124.8564272801036


In [41]:
#Model function
def model(algorithm,dtrain_X,dtrain_Y,dtest_X,dtest_Y,cols=None):

    algorithm.fit(dtrain_X,dtrain_Y)
    predictions = algorithm.predict(dtest_X)
    prediction_probabilities = algorithm.predict(dtest_X)
    print (algorithm)
    
    return predictions,prediction_probabilities,algorithm

In [42]:
y_pred,y_prob,model_obj=model(model_lr,X_train,y_train,X_test,y_test,X.columns)

LinearRegression(copy_X='True', fit_intercept='True', n_jobs=-1)


In [43]:
from mosaicml import *
from mosaicml.constants import MLModelFlavours

In [44]:
@scoring_func
def score(model, request):
    import sanic
    sanic.__file__
    sanic.__version__
    payload = request.json["payload"]
    data = pd.DataFrame(eval(payload))
    prediction = pd.Series(model.predict(data))
    return prediction.tolist()

In [45]:
import requests
req = requests.Request()
req.json = {"payload":X.head(1).to_json()}

In [46]:
import scipy

In [47]:
scipy.__file__

'/opt/conda/lib/python3.7/site-packages/scipy/__init__.py'

In [48]:
scipy.__version__

'1.7.3'

In [49]:
sch = generate_schema(score,(model_obj, req),X)
print(sch)

{'input': {'type': 'pd.core.frame.DataFrame', 'items': {'type': 'object', 'properties': {'carat': {'type': 'number', 'format': 'double'}, 'depth': {'type': 'number', 'format': 'double'}, 'table': {'type': 'number', 'format': 'double'}, 'x': {'type': 'number', 'format': 'double'}, 'y': {'type': 'number', 'format': 'double'}, 'z': {'type': 'number', 'format': 'double'}, 'cut_encoded_Fair': {'type': 'integer', 'format': 'uint8'}, 'cut_encoded_Good': {'type': 'integer', 'format': 'uint8'}, 'cut_encoded_Ideal': {'type': 'integer', 'format': 'uint8'}, 'cut_encoded_Premium': {'type': 'integer', 'format': 'uint8'}, 'cut_encoded_Very Good': {'type': 'integer', 'format': 'uint8'}, 'color_encoded_D': {'type': 'integer', 'format': 'uint8'}, 'color_encoded_E': {'type': 'integer', 'format': 'uint8'}, 'color_encoded_F': {'type': 'integer', 'format': 'uint8'}, 'color_encoded_G': {'type': 'integer', 'format': 'uint8'}, 'color_encoded_H': {'type': 'integer', 'format': 'uint8'}, 'color_encoded_I': {'type

In [50]:
register_model(model_obj, score, name="Sklearn_Model_With_Package", 
                description="Predict Diamond Price",
                flavour=MLModelFlavours.sklearn, schema=sch,
                y_true=y_test, y_pred=pd.Series(y_pred), 
                prob=y_pred, features=X.columns, labels=["price"], init_script="" ,
                model_type="regression", 
                input_type="json", explain_ai=True, x_train=X_train, x_test=X_test, 
                y_train=y_train, y_test=y_test,
                feature_names=list(X.columns),feature_ids=list(X.columns),
                target_names=["price"], 
                kyd=False, kyd_score =False
              )
print("Registering model")

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


Registering model
